In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#clear from non-Moscow objects

#Moscow shape:
#lat: 55.582238 .. 55.905471
#lon: 37.372692 .. 37.851569
#38 х 33 км

fishnet_df = pd.read_csv("fishnet.csv")
fishnet_df.columns = ["lon", "lat", "cell_id", "cell_area", "cell_area2"]
fishnet_df = fishnet_df[["lon", "lat", "cell_id"]]
fishnet_msc_df = fishnet_df[(fishnet_df["lat"]>55.582238) & (fishnet_df["lat"]<55.905471) & (fishnet_df["lon"]>37.372692) & (fishnet_df["lon"]<37.851569)]
fishnet_msc_df

In [ ]:
msc_id = fishnet_msc_df.cell_id.unique().tolist()

In [ ]:
#read Big File №1

dfs = []
for chunk in pd.read_csv("gdrive/My Drive/02_CDensity_1_201903.csv", sep=";", chunksize=100000):
  dfs.append(chunk.loc[chunk['zid'].isin(msc_id)])

In [ ]:
df_part1 = pd.concat(dfs)
df_part1["ts"] = df_part1.ts.apply(lambda x: x.split()[0])
df_part1 = df_part1.groupby(["ts", "zid"]).sum().reset_index().merge(fishnet_msc_df, how="left", left_on="zid", right_on="cell_id")
df_part1.columns = ["ts", "zid", "customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat", "cell_id"]
df_part1 = df_part1[["ts", "zid", "customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat"]]
df_part1["ts"] = pd.to_datetime(df_part1['ts'], format="%m.%d").apply(lambda dt: dt.replace(year=2020))
df_part1

In [ ]:
#read Big File №2

dfs = []
for chunk in pd.read_csv("gdrive/My Drive/02_CDensity_1_201904.csv", sep=";", chunksize=100000):
  dfs.append(chunk.loc[chunk['zid'].isin(msc_id)])

In [ ]:
df_part2 = pd.concat(dfs)
df_part2["ts"] = df_part2.ts.apply(lambda x: x.split()[0])
df_part2 = df_part2.groupby(["ts", "zid"]).sum().reset_index().merge(fishnet_msc_df, how="left", left_on="zid", right_on="cell_id")
df_part2.columns = ["ts", "zid", "customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat", "cell_id"]
df_part2 = df_part2[["ts", "zid", "customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat"]]
df_part2["ts"] = pd.to_datetime(df_part2['ts'], format="%m.%d").apply(lambda dt: dt.replace(year=2020))
df_part2

In [ ]:
pd.concat([df_part1, df_part2]).to_csv("DIT_DATA_AGGRS.csv", index=False)